In [3]:
import pandas as pd
import sqlite3

### yeni verinin eskisiyle aynı olup olmadığını kontrol edildi

In [3]:
PPA_new = pd.read_excel("USA_12.6.22.xlsx", dtype=str)
PPA_old = pd.read_excel("USA_version2.xlsx", dtype=str)

In [4]:
# check if two columns of a dataframe are equal
(PPA_new['BIOID'] == PPA_old['BIOID']).all()

True

### iki veri birleştirildi

burada dsp ve github verisi birleştirildi. code sütunu elle oluşturuldu. 

daha sonra sqlite'a aktarıldı. orada buna göre twitlerin filtrelenmesi yapıldı.

filtrelenen twitlerden hashtagler çekildi ve chatGPT ile kategorileri oluşturuldu

In [20]:
import pandas as pd

# Load and preprocess the PPA dataset
PPA = pd.read_excel('dsp.xlsx', dtype=str)
PPA = PPA.applymap(lambda x: x.strip() if isinstance(x, str) else x)
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maryland': 'MD',
    'Missouri': 'MO',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Washington': 'WA',
    'Wisconsin': 'WI'
}
PPA['state_abb'] = PPA['OFFICE1_CONSTITUENCYLEVEL2'].map(state_abbreviations)
PPA = PPA[
    (PPA['OFFICE1_LEVEL'] == "1") & 
    (PPA['PARTY NAME, ENGLISH'].isin(['Democrat', 'Democrat/NPL', 'Democrat/Working Families', 'Republican', 'Republican/Conservative'])) &
    (PPA['OFFICE1'].isin(['U.S. Senate', 'US Senate', 'U.S. Senator'])) &
    (PPA['OFFICE1_ELECTION_RESULT_GENERAL'].isin(['0', '1']))
]
print(len(PPA))

# Filter the PPA dataset to include only winners
PPA_winners = PPA[PPA['OFFICE1_ELECTION_RESULT_GENERAL'] == "1"]
print(len(PPA_winners))

PPA_losers = PPA[PPA['OFFICE1_ELECTION_RESULT_GENERAL'] == "0"]
print(len(PPA_losers))

# Load and preprocess the github dataset
github = pd.read_excel('githubFixed.xlsx', dtype=str)
github = github[github["type"] == "sen"]
github = github.applymap(lambda x: x.strip() if isinstance(x, str) else x)
print(len(github))


72
35
37
100


In [21]:
# Merge the filtered PPA dataset with the github dataset using full names
merged_data_fullname_winners = pd.merge(PPA_winners, github, 
                                        left_on='code', 
                                        right_on='code', 
                                        how='outer')

# Check the length of the resulting dataset after merging with full names
len(merged_data_fullname_winners)

100

In [22]:
result = pd.concat([merged_data_fullname_winners, PPA_losers], axis=0, ignore_index=True)
len(result)

137

In [23]:
result = result.rename(columns={'gender': 'gender2'})

In [14]:
# connect to sqlite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")

# Convert all columns in the result DataFrame to string type
result = result.astype(str)

# send results to sqlite database
result.to_sql('senatorlist', conn, if_exists='replace', index=False)

# close connection
conn.close()


## hashtagler

### upload birleştirilmiş hastag table to sqlite

In [3]:
#read excel file
df = pd.read_excel('all_hashtags.xlsx', dtype=str)

c:\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
df

,#,hashtag,cluster,frequency,unique_users
0,1,IASen,IASEN,186,3
1,2,NHSEN,NHSEN,127,1
2,3,HurricaneIan,HuracanIan,126,7
3,4,ElectionDay,ElectionDay,126,30
4,5,RonsRoadTour,RonsRoadTour,118,1
...,...,...,...,...,...
1959,1960,20Counties,20Counties,1,1
1960,1961,2023FarmBill,FarmBill,1,1
1961,1962,2022GWSummit,2022GWSummit,1,1
1962,1963,2022CAGeneralElection,2022CAGeneralElection,1,1


In [5]:
# connect to sqlite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")

# send results to sqlite database
df.to_sql('all_hashtags', conn, if_exists='replace', index=False)

# close connection
conn.close()

In [1]:
import sqlite3
import json

# Connect to your SQLite database
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()


In [ ]:
# Fetch entities from the tweets table
cursor.execute("SELECT OFFICE2, entities FROM tumTweets2")
rows = cursor.fetchall()

for row in rows:
    OFFICE2 = row[0]
    entities_data = row[1]

    # Check if entities_data is not None before parsing
    if entities_data:
        try:
            entities_json = json.loads(entities_data)
            
            hashtags = entities_json.get('hashtags', [])
            
            for hashtag in hashtags:
                # Extract the actual hashtag text
                hashtag_text = hashtag['tag']
                
                # Insert into the new table
                cursor.execute("INSERT INTO tweet_hashtags2 (OFFICE2, hashtag) VALUES (?, ?)", (OFFICE2, hashtag_text))
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for author_id: {OFFICE2}")

# Commit changes and close connection
conn.commit()
conn.close()


### expert sup için adjacency

In [3]:
SHEET_ID = '1QjDreFrElDv-Edh235ImxsNv-KVArlWItx19tHMKOYI'

SHEET_NAME = 'all_cats_expert_sup'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
hashtags = pd.read_csv(url)
hashtags = hashtags[['cluster', 'sup_category']]
hashtags

,cluster,sup_category
0,AmericanRescuePlan,Economy
1,BidenFlation,Economy
2,Bidenomics,Economy
3,BidenRecession,Economy
4,CancelStudentDebt,Economy
...,...,...
102,DefendWomensRights,SJ Women
103,Roe,SJ Women
104,RoeRoeRoeYourVote,SJ Women
105,RoevemberIsComing,SJ Women


In [4]:
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()
tweet_hashtags = pd.read_sql_query("SELECT * FROM tweet_hashtags2", conn)
conn.close()

tweet_hashtags


,OFFICE2,hashtag,cluster
0,1,GoDogs,GoDogs
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct
2,1,ShopSmall,ShopSmall
3,1,TransDayOfRemembrance,TransDayOfRemembrance
4,1,AxeFiles,AxeFiles
...,...,...,...
8026,42,Diwali,Diwali
8027,42,BidenBorderCrisis,BidenBorderCrisis
8028,42,GBO,GBO
8029,42,NationalFarmersDay,NationalFarmersDay


In [5]:
#left join hashtags and tweet_hashtags on cluster
merged_data = pd.merge(tweet_hashtags, hashtags,
                                        left_on='cluster',
                                        right_on='cluster',
                                        how='left')
merged_data

,OFFICE2,hashtag,cluster,sup_category
0,1,GoDogs,GoDogs,NaN
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct,NaN
2,1,ShopSmall,ShopSmall,Economy
3,1,TransDayOfRemembrance,TransDayOfRemembrance,SJ Identity
4,1,AxeFiles,AxeFiles,NaN
...,...,...,...,...
8026,42,Diwali,Diwali,NaN
8027,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8028,42,GBO,GBO,NaN
8029,42,NationalFarmersDay,NationalFarmersDay,NaN


In [6]:
# filter where sub_category is not null
merged_data = merged_data[merged_data['sup_category'].notnull()]

merged_data

,OFFICE2,hashtag,cluster,sup_category
2,1,ShopSmall,ShopSmall,Economy
3,1,TransDayOfRemembrance,TransDayOfRemembrance,SJ Identity
5,1,DACA,DACA,Immigration
7,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
8,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
...,...,...,...,...
8017,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8018,42,BidenInflation,BidenFlation,Economy
8019,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8022,42,BidenBorderCrisis,BidenBorderCrisis,Immigration


In [7]:
# pivot table cluster and office2
merged_data.pivot_table(index=['sup_category', 'cluster'], aggfunc='size').reset_index()

,sup_category,cluster,0
0,Economy,AmericanRescuePlan,10
1,Economy,BidenFlation,104
2,Economy,BidenRecession,3
3,Economy,Bidenomics,4
4,Economy,CancelStudentDebt,7
...,...,...,...
102,SJ Women,Roevember,9
103,SJ Women,RoevemberIsComing,3
104,SJ Women,SpeakOutAct,4
105,SJ Women,Women,2


In [8]:
# pivot table cluster and office2
merged_data.pivot_table(index=['sup_category'], aggfunc='size').reset_index()

,sup_category,0
0,Economy,311
1,Environment,45
2,Health,157
3,Immigration,118
4,SJ Identity,184
5,SJ Labor,64
6,SJ Women,74


In [9]:
#get rows where OFFICE2 and cluster both are duplicated
merged_data[merged_data.duplicated(['OFFICE2', 'cluster'], keep=False)]

,OFFICE2,hashtag,cluster,sup_category
5,1,DACA,DACA,Immigration
7,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
8,1,RespectForMarriageAct,RespectForMarriage,SJ Identity
16,1,DACA,DACA,Immigration
53,2,gasprices,gasprices,Economy
...,...,...,...,...
8017,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8018,42,BidenInflation,BidenFlation,Economy
8019,42,BidenBorderCrisis,BidenBorderCrisis,Immigration
8022,42,BidenBorderCrisis,BidenBorderCrisis,Immigration


In [10]:

#drop duplicates in cluster
merged_data = merged_data.drop_duplicates(subset=['OFFICE2', 'cluster'], keep='first')

In [11]:
#filter merged_data where sup_category is Economy, Environment, Health, Immigration, SJ Identity, SJ Labor, SJ Women
sup_econ = merged_data[merged_data['sup_category'] == 'Economy']
sup_env = merged_data[merged_data['sup_category'] == 'Environment']
sup_health = merged_data[merged_data['sup_category'] == 'Health']
sup_imm = merged_data[merged_data['sup_category'] == 'Immigration']
sup_sjid = merged_data[merged_data['sup_category'] == 'SJ Identity']
sup_sjlab = merged_data[merged_data['sup_category'] == 'SJ Labor']
sup_sjwom = merged_data[merged_data['sup_category']== 'SJ Women']

In [12]:
sup_econ

,OFFICE2,hashtag,cluster,sup_category
2,1,ShopSmall,ShopSmall,Economy
14,1,DemsCreateJobs,DemsCreateJobs,Economy
53,2,gasprices,gasprices,Economy
98,4,SmallBusinessSaturday,SmallBusinessSaturday,Economy
100,4,CancelStudentDebt,CancelStudentDebt,Economy
...,...,...,...,...
7678,83,SmallBusinessSaturday,SmallBusinessSaturday,Economy
7826,83,ManufacturingMonth,ManufacturingDay,Economy
7965,63,SmallBusinessSaturday,SmallBusinessSaturday,Economy
7983,42,Bidenflation,BidenFlation,Economy


In [13]:
import pandas as pd
import numpy as np
import sqlite3  # Ensure you import sqlite3

def generate_adjacency(df):
    df = df.copy()
    # Convert OFFICE2 column to integers
    df['OFFICE2'] = df['OFFICE2'].astype(int)
    
    # Given OFFICE2 is always between 1-137
    users = np.arange(1, 138)
    
    # Initialize a 137x137 matrix with zeros
    matrix = np.zeros((137, 137))
    
    # For each unique hashtag
    for cluster in df['cluster'].unique():
        print(f"Processing cluster: {cluster}")  # Print current cluster
        
        # Get list of users who used this hashtag
        users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
        
        # Update the matrix for every pair of these users
        for i in range(len(users_with_hashtag)):
            for j in range(i+1, len(users_with_hashtag)):
                # Get matrix indices for the users
                user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
                user_j_idx = users_with_hashtag[j] - 1
                
                # Check if the indices are within bounds
                if 0 <= user_i_idx < 138 and 0 <= user_j_idx < 138:
                    # Set the matrix entries for this pair of users to 1
                    matrix[user_i_idx][user_j_idx] = 1
                    matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
                else:
                    print(f"WARNING: Detected out of bounds OFFICE2 value. Skipping pair ({users_with_hashtag[i]}, {users_with_hashtag[j]})")
                
    # Convert the matrix to a DataFrame
    return pd.DataFrame(matrix, index=users, columns=users)


def save_to_sqlite(df, db_name, table_name):
    print(f"Saving DataFrame to SQLite with table name: {table_name}")  # Print current saving status
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace')
    conn.close()

# Create a dictionary mapping DataFrame names to DataFrames
df_dict = {
    'sup_econ': sup_econ,
    'sup_env': sup_env,
    'sup_health': sup_health,
    'sup_imm': sup_imm,
    'sup_sjid': sup_sjid,
    'sup_sjlab': sup_sjlab,
    'sup_sjwom': sup_sjwom
}

for name, df in df_dict.items():
    print(f"Generating adjacency matrix for DataFrame: {name}")  # Print current DataFrame being processed
    adjacency_df = generate_adjacency(df)
    save_to_sqlite(adjacency_df, "csvyerine2.db", name)  # Use the name directly as the table name

print("All done!")  # Print completion message


Generating adjacency matrix for DataFrame: sup_econ
Processing cluster: ShopSmall
Processing cluster: DemsCreateJobs
Processing cluster: gasprices
Processing cluster: SmallBusinessSaturday
Processing cluster: CancelStudentDebt
Processing cluster: ManufacturingDay
Processing cluster: ChildTaxCredit
Processing cluster: InflationReductionAct
Processing cluster: BidenFlation
Processing cluster: inflation
Processing cluster: SmallBizSatuday
Processing cluster: FarmBill
Processing cluster: SmallBiz
Processing cluster: ShopLocal
Processing cluster: BidenRecession
Processing cluster: SocialSecurity
Processing cluster: Bidenomics
Processing cluster: AmericanRescuePlan
Processing cluster: IRS
Processing cluster: SPR
Processing cluster: jobs
Saving DataFrame to SQLite with table name: sup_econ
Generating adjacency matrix for DataFrame: sup_env
Processing cluster: DARTMission
Processing cluster: ForestProductsWeek
Processing cluster: TreeOfLife
Processing cluster: energy
Processing cluster: CleanW

In [13]:

import numpy as np

# Sample data
data = {
    'OFFICE2': [13, 4, 2, 5, 1],
    'cluster': ['ShopSmall', 'DemsCreateJobs', 'gasprices', 'gasprices', 'gasprices']
}

df = pd.DataFrame(data)

df = df.copy()

# Convert OFFICE2 column to integers
df['OFFICE2'] = df['OFFICE2'].astype(int)

# Given OFFICE2 is always between 1-137
users = np.arange(1, 138)

# Initialize a 128x128 matrix with zeros
matrix = np.zeros((137, 137))

# For each unique hashtag
for cluster in df['cluster'].unique():
    print(f"Processing cluster: {cluster}")  # Print current cluster
    
    # Get list of users who used this hashtag
    users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
    
    # Update the matrix for every pair of these users
    for i in range(len(users_with_hashtag)):
        for j in range(i+1, len(users_with_hashtag)):
            # Get matrix indices for the users
            user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
            user_j_idx = users_with_hashtag[j] - 1
            
            # Check if the indices are within bounds
            if 0 <= user_i_idx < 138 and 0 <= user_j_idx < 138:
                # Set the matrix entries for this pair of users to 1
                matrix[user_i_idx][user_j_idx] = 1
                matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
            else:
                print(f"WARNING: Detected out of bounds OFFICE2 value. Skipping pair ({users_with_hashtag[i]}, {users_with_hashtag[j]})")
            
# Convert the matrix to a DataFrame
matrix = pd.DataFrame(matrix, index=users, columns=users)
matrix

Processing cluster: ShopSmall
Processing cluster: DemsCreateJobs
Processing cluster: gasprices


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [14]:
# meta datayı gönderelim
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()
#print all table names in sqlite
# Execute the SQL command to fetch all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Loop through the results and print table names
for table in tables:
    print(table[0])

sqlite_sequence
githubTweets
allPPATweets
senatorlist
allPPATweetsFiltered
githubTweetsFiltered
tumTweets
tumTweets2
all_hashtags
tweet_hashtags
hashtag_summary
tweet_hashtags2


### metadata gönder

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("D:/Mega/polarization/twitter004.db")
cursor = conn.cursor()
senators = pd.read_sql_query("SELECT * FROM senatorlist", conn)
conn.close()

senators

In [4]:
#save senators df to sqlite
conn = sqlite3.connect("csvyerine2.db")
senators.to_sql('metadata', conn, if_exists='replace', index=False)
conn.close()

### unsup expert için adjacency

In [17]:
SHEET_ID = '1QjDreFrElDv-Edh235ImxsNv-KVArlWItx19tHMKOYI'

SHEET_NAME = 'all_cats_unsup_expert'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
hashtags = pd.read_csv(url)
hashtags = hashtags[['cluster', 'Expert_Label2']]
hashtags

,cluster,Expert_Label2
0,ThankAFarmer,Awareness & Appreciation
1,FirstResponder,Awareness & Appreciation
2,buylocal,Awareness & Appreciation
3,USMC247,Awareness & Appreciation
4,ShopLocal,Awareness & Appreciation
...,...,...
400,USSenateDebateFL,State Politcs
401,VegasStrong,State Politcs
402,VoteOutMarkKelly,State Politcs
403,WASEN,State Politcs


In [18]:
conn = sqlite3.connect("D:/Dropbox/projeler/ndsu_polarization/twitterVeriToplama/twitter004.db")
cursor = conn.cursor()
tweet_hashtags = pd.read_sql_query("SELECT * FROM tweet_hashtags2", conn)
conn.close()

tweet_hashtags


,OFFICE2,hashtag,cluster
0,1,GoDogs,GoDogs
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct
2,1,ShopSmall,ShopSmall
3,1,TransDayOfRemembrance,TransDayOfRemembrance
4,1,AxeFiles,AxeFiles
...,...,...,...
8026,42,Diwali,Diwali
8027,42,BidenBorderCrisis,BidenBorderCrisis
8028,42,GBO,GBO
8029,42,NationalFarmersDay,NationalFarmersDay


In [19]:
#left join hashtags and tweet_hashtags on cluster
merged_data = pd.merge(tweet_hashtags, hashtags,
                                        left_on='cluster',
                                        right_on='cluster',
                                        how='left')
merged_data

,OFFICE2,hashtag,cluster,Expert_Label2
0,1,GoDogs,GoDogs,NaN
1,1,VeteranServiceRecognitionAct,VeteranServiceRecognitionAct,NaN
2,1,ShopSmall,ShopSmall,Awareness & Appreciation
3,1,TransDayOfRemembrance,TransDayOfRemembrance,Awareness & Appreciation
4,1,AxeFiles,AxeFiles,Media
...,...,...,...,...
8050,42,Diwali,Diwali,Awareness & Appreciation
8051,42,BidenBorderCrisis,BidenBorderCrisis,Governance
8052,42,GBO,GBO,Media
8053,42,NationalFarmersDay,NationalFarmersDay,Awareness & Appreciation


In [21]:
# filter where sub_category is not null
merged_data = merged_data[merged_data['Expert_Label2'].notnull()]

merged_data

,OFFICE2,hashtag,cluster,Expert_Label2
2,1,ShopSmall,ShopSmall,Awareness & Appreciation
3,1,TransDayOfRemembrance,TransDayOfRemembrance,Awareness & Appreciation
4,1,AxeFiles,AxeFiles,Media
5,1,DACA,DACA,Governance
7,1,RespectForMarriageAct,RespectForMarriage,Rights & Social Issues
...,...,...,...,...
8049,42,NationalFirstRespondersDay,NationalFirstRespondersDay,Awareness & Appreciation
8050,42,Diwali,Diwali,Awareness & Appreciation
8051,42,BidenBorderCrisis,BidenBorderCrisis,Governance
8052,42,GBO,GBO,Media


In [22]:
# pivot table cluster and office2
merged_data.pivot_table(index=['Expert_Label2', 'cluster'], aggfunc='size').reset_index()

,Expert_Label2,cluster,0
0,Awareness & Appreciation,4Hweek,3
1,Awareness & Appreciation,AlaskaDay,3
2,Awareness & Appreciation,AmericanDiabetesMonth,2
3,Awareness & Appreciation,AmericanEducationWeek,5
4,Awareness & Appreciation,ArmyNavy,2
...,...,...,...
397,State Politcs,WASEN,58
398,State Politcs,WISen,24
399,State Politcs,arpx,6
400,State Politcs,copolitics,4


In [25]:
#get rows where OFFICE2 and cluster both are duplicated
merged_data[merged_data.duplicated(['OFFICE2', 'cluster'], keep=False)]

,OFFICE2,hashtag,cluster,Expert_Label2
4,1,AxeFiles,AxeFiles,Media
5,1,DACA,DACA,Governance
7,1,RespectForMarriageAct,RespectForMarriage,Rights & Social Issues
8,1,RespectForMarriageAct,RespectForMarriage,Rights & Social Issues
16,1,DACA,DACA,Governance
...,...,...,...,...
8043,42,BidenBorderCrisis,BidenBorderCrisis,Governance
8046,42,BidenBorderCrisis,BidenBorderCrisis,Governance
8047,42,GBO,GBO,Media
8051,42,BidenBorderCrisis,BidenBorderCrisis,Governance


In [26]:

#drop duplicates in cluster
merged_data = merged_data.drop_duplicates(subset=['OFFICE2', 'cluster'], keep='first')
len(merged_data)

1626

In [27]:
# pivot table cluster and office2
merged_data.pivot_table(index=['Expert_Label2'], aggfunc='size').reset_index()

,Expert_Label2,0
0,Awareness & Appreciation,690
1,Campaign,202
2,Governance,280
3,Media,164
4,Rights & Social Issues,118
5,State Politcs,172


In [28]:
#filter merged_data where sup_category is Economy, Environment, Health, Immigration, SJ Identity, SJ Labor, SJ Women
unsup_awe = merged_data[merged_data['Expert_Label2'] == 'Awareness & Appreciation']
unsup_cam = merged_data[merged_data['Expert_Label2'] == 'Campaign']
unsup_gov = merged_data[merged_data['Expert_Label2'] == 'Governance']
unsup_med = merged_data[merged_data['Expert_Label2'] == 'Media']
unsup_rig = merged_data[merged_data['Expert_Label2'] == 'Rights & Social Issues']
unsup_sta = merged_data[merged_data['Expert_Label2']== 'State Politcs']

In [29]:
import pandas as pd
import numpy as np
import sqlite3  # Ensure you import sqlite3

def generate_adjacency(df):
    df = df.copy()
    # Convert OFFICE2 column to integers
    df['OFFICE2'] = df['OFFICE2'].astype(int)
    
    # Given OFFICE2 is always between 1-137
    users = np.arange(1, 138)
    
    # Initialize a 137x137 matrix with zeros
    matrix = np.zeros((137, 137))
    
    # For each unique hashtag
    for cluster in df['cluster'].unique():
        print(f"Processing cluster: {cluster}")  # Print current cluster
        
        # Get list of users who used this hashtag
        users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
        
        # Update the matrix for every pair of these users
        for i in range(len(users_with_hashtag)):
            for j in range(i+1, len(users_with_hashtag)):
                # Get matrix indices for the users
                user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
                user_j_idx = users_with_hashtag[j] - 1
                
                # Check if the indices are within bounds
                if 0 <= user_i_idx < 138 and 0 <= user_j_idx < 138:
                    # Set the matrix entries for this pair of users to 1
                    matrix[user_i_idx][user_j_idx] = 1
                    matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
                else:
                    print(f"WARNING: Detected out of bounds OFFICE2 value. Skipping pair ({users_with_hashtag[i]}, {users_with_hashtag[j]})")
                
    # Convert the matrix to a DataFrame
    return pd.DataFrame(matrix, index=users, columns=users)


def save_to_sqlite(df, db_name, table_name):
    print(f"Saving DataFrame to SQLite with table name: {table_name}")  # Print current saving status
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace')
    conn.close()

# Create a dictionary mapping DataFrame names to DataFrames
df_dict = {
    'unsup_awe': unsup_awe,
    'unsup_cam': unsup_cam,
    'unsup_gov': unsup_gov,
    'unsup_med': unsup_med,
    'unsup_rig': unsup_rig,
    'unsup_sta': unsup_sta
}

for name, df in df_dict.items():
    print(f"Generating adjacency matrix for DataFrame: {name}")  # Print current DataFrame being processed
    adjacency_df = generate_adjacency(df)
    save_to_sqlite(adjacency_df, "csvyerine2.db", name)  # Use the name directly as the table name

print("All done!")  # Print completion message


Generating adjacency matrix for DataFrame: unsup_awe
Processing cluster: ShopSmall
Processing cluster: TransDayOfRemembrance
Processing cluster: VeteransDay
Processing cluster: Diwali
Processing cluster: FirePreventionWeek
Processing cluster: IndigenousPeopleDay
Processing cluster: HispanicHeritageMonth
Processing cluster: FilipinoAmericanHistoryMonth
Processing cluster: LatinaEqualPayDay
Processing cluster: WorldAIDSDay
Processing cluster: NativeAmericanHeritageDay
Processing cluster: Thanksgiving
Processing cluster: NevadaDay
Processing cluster: FightRetailCrimeDay
Processing cluster: BreastCancerAwarenessMonth
Processing cluster: NationalWildlifeRefugeWeek
Processing cluster: NationalMentalIllnessAwarenessWeek
Processing cluster: DomesticViolenceAwarenessMonth
Processing cluster: SmallBusinessSaturday
Processing cluster: DiaDeLosMuertos
Processing cluster: NationalComingOutDay
Processing cluster: ManufacturingDay
Processing cluster: AmericanEducationWeek
Processing cluster: WorldFre

In [ ]:

import numpy as np

# Sample data
data = {
    'OFFICE2': [13, 4, 2, 5, 1],
    'cluster': ['ShopSmall', 'DemsCreateJobs', 'gasprices', 'gasprices', 'gasprices']
}

df = pd.DataFrame(data)

df = df.copy()

# Convert OFFICE2 column to integers
df['OFFICE2'] = df['OFFICE2'].astype(int)

# Given OFFICE2 is always between 1-137
users = np.arange(1, 138)

# Initialize a 128x128 matrix with zeros
matrix = np.zeros((137, 137))

# For each unique hashtag
for cluster in df['cluster'].unique():
    print(f"Processing cluster: {cluster}")  # Print current cluster
    
    # Get list of users who used this hashtag
    users_with_hashtag = df[df['cluster'] == cluster]['OFFICE2'].tolist()
    
    # Update the matrix for every pair of these users
    for i in range(len(users_with_hashtag)):
        for j in range(i+1, len(users_with_hashtag)):
            # Get matrix indices for the users
            user_i_idx = users_with_hashtag[i] - 1  # subtract 1 since array indices start from 0
            user_j_idx = users_with_hashtag[j] - 1
            
            # Check if the indices are within bounds
            if 0 <= user_i_idx < 138 and 0 <= user_j_idx < 138:
                # Set the matrix entries for this pair of users to 1
                matrix[user_i_idx][user_j_idx] = 1
                matrix[user_j_idx][user_i_idx] = 1  # Symmetrical matrix
            else:
                print(f"WARNING: Detected out of bounds OFFICE2 value. Skipping pair ({users_with_hashtag[i]}, {users_with_hashtag[j]})")
            
# Convert the matrix to a DataFrame
matrix = pd.DataFrame(matrix, index=users, columns=users)
matrix

In [30]:
#close connection
conn.close()

## tekli networkler

şimdilik yapılmayacak